In [1]:
import os
os.chdir("../")
%pwd

'/media/Ubuntu/meg/mlprojects/Semantic_Segmentation_for_self_driving_cars'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir:Path
    checkpoint_model_filepath: Path

In [3]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [4]:
from box import ConfigBox
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

2023-09-24 17:45:12.743951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 17:45:13.349198: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
class ConfigurationManager:
    def __init__(
            self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH) :
                 
        self.config = read_yaml(config_filepath),
        self.params = read_yaml(params_filepath),
        self.config= ConfigBox(self.config[0]) # configbox is returning tuble so to avoid it we are again applying configbox to the dictionary at index 0
        self.params= ConfigBox(self.params[0])



    def get_prepare_callback_config(self)-> PrepareCallbackConfig:
        config = self.config.prepare_callbacks

        create_directories([config.root_dir])


        create_directories([ 
                            Path(config.checkpoint_model_filepath),
                            Path(config.tensorboard_root_log_dir)  ])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir = Path(config.root_dir),
            tensorboard_root_log_dir = Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath= Path(config.checkpoint_model_filepath)
    
        )

        return prepare_callback_config
    


In [12]:
class PrepareCallback:
    def __init__(self, config : PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}",)
        print(tb_running_log_dir)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    
    @property
    def _create_ckpt_callbacks(self):
        
        print(self.config.checkpoint_model_filepath)

        return tf.keras.callbacks.ModelCheckpoint(
            filepath= os.path.join(self.config.checkpoint_model_filepath,"weights-{epoch:03d}-{val_loss:.4f}.hdf5"),
            save_best_only=True)
    
    
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_ckpt_callbacks]


In [13]:
try: 
    config = ConfigurationManager()
    prepare_callbacks_config= config.get_prepare_callback_config()
    prepare_callbacks= PrepareCallback(config = prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

except Exception as e:
    raise e

[2023-09-24 18:09:13,296: INFO: common: yaml fileconfig.yamlloaded Successfully]
[2023-09-24 18:09:13,299: INFO: common: yaml fileparams.yamlloaded Successfully]
[2023-09-24 18:09:13,299: INFO: common: created directory at:artifacts/callbacks]
[2023-09-24 18:09:13,300: INFO: common: created directory at:artifacts/callbacks/checkpoints]
[2023-09-24 18:09:13,301: INFO: common: created directory at:artifacts/callbacks/tensorboard_log_dir]
artifacts/callbacks/tensorboard_log_dir/tb_logs_at_2023-09-24-18-09-13
artifacts/callbacks/checkpoints
